# Data Processing
This notebook is meant to recreate the data cleaing process from evolving hockey but using python
https://evolving-hockey.com/blog/a-new-expected-goals-model-for-predicting-goals-in-the-nhl/

### Update 1
For this update I am removing some columns that would result in multicollinearity. 
I will also be breakup up game period into seperate dummy variables. 
This shouldn't effects the performance to much, but is more "proper"

In [4]:
#Libraries used
import pandas as pd #for the data frame
import numpy as np #used for filtering and some arithmatic. 

In [5]:
def clean_my_data(season):
    
    #These two lists are used for different event types that will be filtered. 
    fenwick = ['SHOT', 'MISS', 'GOAL']
    corsi = ['SHOT', 'MISS', 'GOAL', 'BLOCK']

    #This is a list that is used to correct some shot type descriptions
    correction = ['Tip-In', 'Wrap-around', 'Deflected']

    #A list of strings that indicate if the teams are even strength
    even_strength = ['5v5','4v4','3v3']

    def get_data(season):
        ''' User inputs the last 2 numbers of the year the playoffs are in for the season of interest
            IE the 2020-2021 season would be 21. 
            This returns the raw data in a dataframe
        '''
        print(f"Collecting the data for 20{season}")
        sea_1 = str(season-1).zfill(2)
        sea_2 = str(season).zfill(2)
        df = pd.read_csv(f'data/pbp/raw/pbp_query_20{sea_1}20{sea_2}.csv',low_memory=False) 
        fen_eve = df[df.event_type.isin(['GOAL','SHOT','MISS'])].shape[0]
        print(f'There are {fen_eve} fenwick events in the raw data')

        return df

    def get_dist_angle(df):
        print("Adding a column for shot distance and angle")
        fen_eve = df[df.event_type.isin(['GOAL','SHOT','MISS'])].shape[0]
        print(f'There are {fen_eve} fenwick events in the raw data')
        #Creating the distance and angle columns and deriving the results from the data
        df['event_distance'] = ((89 - abs(df['coords_x']))**2 + df['coords_y']**2)**(1/2)
        df['event_angle'] = abs(np.arctan(df['coords_y'] / (89 - abs(df['coords_x']))) * (180 / np.pi))
        return df

    def get_event_zone(df):
        print("Collecting the even zone from the description and making corrections as needed")
        fen_eve = df[df.event_type.isin(['GOAL','SHOT','MISS'])].shape[0]
        print(f'There are {fen_eve} fenwick events in the raw data')
        #Setting up the variables to be used to extract the zone from the description
        col = 'event_description'
        conditions = [df[col].str.contains('Off. Zone', na=False), df[col].str.contains('Def. Zone', na=False), 
                      df[col].str.contains('Neu. Zone', na=False)]
        choices = ['Off','Def','Neu']

        #Creating The event zone column
        df['event_zone'] = np.select(conditions, choices, default=np.nan)
        df['event_zone'] = np.where((df.event_zone == 'Def') & 
                                          (df.event_type == 'BLOCK'), 
                                          'Off',df.event_zone)
        df['event_zone'] = np.where((df.event_type.isin(fenwick)) & 
                                          (df.event_zone == 'Def') & 
                                          (df.pbp_distance <= 64),
                                          'Off',df.event_zone)

        return df

    def long_shot_correction(df):
        print("Making corrections for shots from outside the offensive zone")
        #This is a correction for if an event is behind the net or actually outside of the offensive zone
        df['event_distance'] = np.where((df.event_type.isin(fenwick)) & 
                                             (df.pbp_distance > 89) &
                                             (df.coords_x < 0) &
                                             (~df.event_detail.isin(correction)) &~
                                             ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                             (((abs(df.coords_x)+89)**2) + df.coords_y**2)**(1/2)
                                             ,df.event_distance)
        df['event_distance'] = np.where((df.event_type.isin(fenwick)) & 
                                             (df.pbp_distance > 89) &
                                             (df.coords_x > 0) &
                                             (~df.event_detail.isin(correction)) &~
                                             ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                             (((df.coords_x+89)**2) + df.coords_y**2)**(1/2)
                                             ,df.event_distance)
        df['event_angle'] = np.where((df.event_type.isin(fenwick)) & 
                                             (df.pbp_distance > 89) &
                                             (df.coords_x < 0) &
                                             (~df.event_detail.isin(correction)) &~
                                             ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                             abs(np.arctan(df['coords_y']/(abs(df['coords_x'])+89))*(180 / np.pi))
                                             ,df.event_angle)
        df['event_angle'] = np.where((df.event_type.isin(fenwick)) & 
                                             (df.pbp_distance > 89) &
                                             (df.coords_x > 0) &
                                             (~df.event_detail.isin(correction)) &~
                                             ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                             abs(np.arctan(df['coords_y']/(df['coords_x']+89))*(180 / np.pi))
                                             ,df.event_angle)
        df['event_zone'] = np.where((df.event_type.isin(fenwick)) & 
                                             (df.event_zone == 'Def') &
                                             (df.pbp_distance <= 64),
                                             'Off',
                                             df.event_zone)

        return df

    def account_for_penaltyshot(df):
        print("Correctly labeling events that are penalty shots")
        #If the event is a penalty shot, it needs to be corrected
        df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                             (df.event_team == df.home_team),
                                             'Ev1',
                                             df.game_strength_state)
        df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                             (df.event_team == df.away_team),
                                             '1vE',
                                             df.game_strength_state)
        df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                             (df.event_team == df.home_team),
                                             1,
                                             df.home_skaters)
        df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                             (df.event_team == df.away_team),
                                             0,
                                             df.home_skaters)
        df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                             (df.event_team == df.home_team),
                                             0,
                                             df.away_skaters)
        df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                             (df.event_team == df.away_team),
                                             1,
                                             df.away_skaters)

        return df

    def first_filter_round(df):
        print("Applying the first round of filtering")
        ''' There are 5 filters being applied in this function
            1. Make sure that event type is a an event of interest
            2. Make sure the event is from the game (period 0 and 6 are pre and post game)
            3. Remove penalty shots
            4. This filter removes edge cases
            5. Remove data points where the position is not known
        '''
        event_lst = ["FAC", "GOAL", "BLOCK", "SHOT", "MISS", "HIT", "TAKE", "GIVE"]
        event_fil = df.event_type.isin(event_lst)
        game_fil  = df.game_period < 5
        pen_fil   = ~df.event_description.str.contains('penalty shot', na=False)
        last_fil  = ~((df.coords_x == 0) &
                     (df.coords_y == 0) &
                     (df.event_type.isin(corsi)) &
                     ((df.pbp_distance != 90) & (df.event_type.isin(fenwick))))
        df = df[event_fil & game_fil & pen_fil & last_fil].copy()
        df.dropna(subset=['coords_x','coords_y'],inplace=True)

        return df

    def get_prev_event(df):
        print("Collecting previous event data")
        ''' This function is the first step in some feature engineering
            To derive insights from how a previous event effects the goal, we must get the previous event
            In order to do this all the events  have to be in order by play, period, game, and season
        '''
        df.sort_values(['season', 'game_id', 'game_period','event_index'],inplace=True)
        df['seconds_since_last'] = df.game_seconds - df.game_seconds.shift(1)
        df['event_type_last'] = df.event_type.shift(1)
        df['event_team_last'] = df.event_team.shift(1)
        df['event_strength_last'] = df.game_strength_state.shift(1)
        df['coords_x_last'] = df.coords_x.shift(1)
        df['coords_y_last'] = df.coords_y.shift(1)

        return df

    def second_filter_round(df):
        print("Applying second round of filtering")
        ''' For this second round of filtering there are 3 filters
            1. Only want fenwick events
            2. Only want even strength data
            3. Remove data points where the position is not known
        '''
        fen_fil  = df.event_type.isin(fenwick)
        even_fil = df.game_strength_state.isin(even_strength)
        df = df[fen_fil & even_fil].copy()
        df.dropna(subset=['coords_x','coords_y'],inplace=True)

        return df

    def get_odds_ends(df):
        print("Adding some odds and ends features")
        ''' A few more features to add that will either directly or indirectly be used
            Was the event from the same team, are they home, what is the score at the time of event
            change some shot types to wrist shots, and the distance from the last event.
            Then to rename events to shots
        '''
        df['same_team_last']     = np.where(df.event_team == df.event_team_last,1,0)
        df['is_home']            = np.where(df.event_team == df.home_team,1,0)
        df['score_state']        = np.where(df.is_home == 1, 
                                            df.home_score - df.away_score, 
                                            df.away_score - df.home_score)
        df['event_detail']       = np.where(df.event_detail.isna(), 'Wrist', df.event_detail)
        df['distance_from_last'] = (((df.coords_x - df.coords_x_last)**2) + 
                                    ((df.coords_y - df.coords_y_last)**2))**(1/2)
        df.rename(columns={"event_distance": "shot_distance", "event_angle": "shot_angle"}, inplace=True)

        return df

    def create_dummy_vars(df):
        print("Creating dummy variables")
        ''' This is creating dummy variables for all the categorical data.
            I'm not happy its done by hand, its not the most elegent, could clean this up in the future
            I am also just returning the final df here
        '''
        df['is_goal'] = np.where(df.event_type == 'GOAL',1,0)
        df['state_5v5'] = np.where(df.game_strength_state == '5v5',1,0)
        df['state_5v4'] = np.where(df.game_strength_state == '5v4',1,0)
        df['state_5v3'] = np.where(df.game_strength_state == '5v3',1,0)
        df['score_down_4'] = np.where(df.score_state <= 4,1,0)
        df['score_down_3'] = np.where(df.score_state == 3,1,0)
        df['score_down_2'] = np.where(df.score_state == 2,1,0)
        df['score_down_1'] = np.where(df.score_state == 1,1,0)
        df['score_even'] = np.where(df.score_state == 0,1,0)
        df['score_up_1'] = np.where(df.score_state == 1,1,0)
        df['score_up_2'] = np.where(df.score_state == 2,1,0)
        df['score_up_3'] = np.where(df.score_state == 3,1,0)
        df['score_up_4'] = np.where(df.score_state >= 4,1,0)
        df['wrist_shot'] = np.where(df.event_detail == 'Wrist',1,0)
        df['deflected_shot'] = np.where(df.event_detail == 'Deflected',1,0)
        df['tip_shot'] = np.where(df.event_detail == 'Tip-In',1,0)
        df['slap_shot'] = np.where(df.event_detail == 'Slap',1,0)
        df['backhand_shot'] = np.where(df.event_detail == 'Backhand',1,0)
        df['snap_shot'] = np.where(df.event_detail == 'Snap',1,0)
        df['wrap_shot'] = np.where(df.event_detail == 'Wrap-around',1,0)
        df['prior_shot_same'] = np.where((df.event_type_last == 'SHOT') & (df.same_team_last == 1),1,0)
        df['prior_miss_same'] = np.where((df.event_type_last == 'MISS') & (df.same_team_last == 1),1,0)
        df['prior_block_same'] = np.where((df.event_type_last == 'BLOCK') & (df.same_team_last == 1),1,0)
        df['prior_shot_opp'] = np.where((df.event_type_last == 'SHOT') & (df.same_team_last == 0),1,0)
        df['prior_miss_opp'] = np.where((df.event_type_last == 'MISS') & (df.same_team_last == 0),1,0)
        df['prior_block_opp'] = np.where((df.event_type_last == 'BLOCK') & (df.same_team_last == 0),1,0)
        df['prior_give_opp'] = np.where((df.event_type_last == 'GIVE') & (df.same_team_last == 0),1,0)
        df['prior_give_same'] = np.where((df.event_type_last == 'GIVE') & (df.same_team_last == 1),1,0)
        df['prior_take_opp'] = np.where((df.event_type_last == 'TAKE') & (df.same_team_last == 0),1,0)
        df['prior_take_same'] = np.where((df.event_type_last == 'TAKE') & (df.same_team_last == 1),1,0)
        df['prior_hit_opp'] = np.where((df.event_type_last == 'HIT') & (df.same_team_last == 0),1,0)
        df['prior_hit_same'] = np.where((df.event_type_last == 'HIT') & (df.same_team_last == 1),1,0)
        df['prior_face'] = np.where(df.event_type_last == 'FAC',1,0)

        final_lst = ['game_period', 'game_seconds','is_home','coords_x', 'coords_y',
                     'coords_x_last', 'coords_y_last','shot_distance', 'shot_angle',
                     'state_5v5', 'state_5v4', 'score_down_4', 'score_down_3',
                     'score_down_2', 'score_down_1', 'score_even', 'score_up_1',
                     'score_up_2', 'score_up_3', 'wrist_shot',
                     'deflected_shot', 'tip_shot', 'slap_shot', 'backhand_shot', 'snap_shot',
                     'prior_shot_same', 'prior_miss_same', 'prior_shot_opp', 'prior_miss_opp',
                     'prior_give_opp', 'prior_take_opp', 'prior_hit_opp', 'prior_face',
                     'state_5v3', 'score_up_4', 'wrap_shot', 'prior_block_same',
                     'prior_block_opp', 'prior_give_same', 'prior_take_same', 'prior_hit_same',
                     'is_goal']

        df = df[final_lst].copy()

        return df

    df_import = get_data(season)
    df_dist_angle = get_dist_angle(df_import)
    df_event_zone = get_event_zone(df_dist_angle)
    df_correction = long_shot_correction(df_event_zone)
    df_penshot    = account_for_penaltyshot(df_correction)
    df_filter_one = first_filter_round(df_penshot)
    df_add_event  = get_prev_event(df_filter_one)
    df_filter_two = second_filter_round(df_add_event)
    df_other      = get_odds_ends(df_filter_two)
    df_final      = create_dummy_vars(df_other)
    print(df_final.shape[0])
    return df_final    
    

In [6]:
lst = [11,12,13,14,15,16,17,18,19,20,21]
for num in lst:
    df = clean_my_data(num)
    df.to_csv(f'data/pbp/ready_to_model/{num-1}_{num}_pbp_cleaned.csv')

There are 104644 fenwick events in the raw data
Adding a column for shot distance and angle
There are 104644 fenwick events in the raw data
There are 104644 fenwick events in the raw data
Making corrections for shots from outside the offensive zone
Correctly labeling events that are penalty shots
Applying the first round of filtering
Applying second round of filtering
Adding some odds and ends features
Creating dummy variables
81727
There are 102871 fenwick events in the raw data
Adding a column for shot distance and angle
There are 102871 fenwick events in the raw data
There are 102871 fenwick events in the raw data
Making corrections for shots from outside the offensive zone
Correctly labeling events that are penalty shots
Applying the first round of filtering
Applying second round of filtering
Adding some odds and ends features
Creating dummy variables
81469
There are 59063 fenwick events in the raw data
Adding a column for shot distance and angle
There are 59063 fenwick events in t